In [ ]:
import fileinput
import glob
import json
import os
import re
import subprocess
import sys

## Data

In [ ]:
def read_mut_data(path): 
    input_file = open(path, "r")
    project_targets = json.loads(input_file.read())
    input_file.close()
    return project_targets

In [ ]:
# temporarily
targets = [
    {
        "mut": "getGlyph(int)",
        "mut-start-line": 155,
        "mut-end-line": 205,
        "parent-module-path": "/home/dee/dev/rick-major-review/pdfbox/fontbox/",
        "parent-file": "GlyphTable.java",
        "parent-fqn": "org.apache.fontbox.ttf.GlyphTable",
        "parent-path": "/home/dee/dev/rick-major-review/pdfbox/fontbox/src/main/java/org/apache/fontbox/ttf/",
        "has-oo": False,
        "test-file": "TestGlyphTableRickGen.java",
        "test-path": "/home/dee/dev/rick-major-review/pdfbox/rick-major/src/test/java/org/apache/fontbox/ttf/",
        "test-module-path": "/home/dee/dev/rick-major-review/pdfbox/rick-major/"
        
    },
    {
        "mut": "decode(InputStream,OutputStream,COSDictionary,int,DecodeOptions)",
        "mut-start-line": 55,
        "mut-end-line": 163,
        "parent-module-path": "/home/dee/dev/rick-major-review/pdfbox/pdfbox/",
        "parent-file": "DCTFilter.java",
        "parent-fqn": "org.apache.pdfbox.filter.DCTFilter",
        "parent-path": "/home/dee/dev/rick-major-review/pdfbox/pdfbox/src/main/java/org/apache/pdfbox/filter/",
        "has-oo": False,
        "test-file": "TestDCTFilterRickGen.java",
        "test-path": "/home/dee/dev/rick-major-review/pdfbox/rick-major/src/test/java/org/apache/pdfbox/filter/",
        "test-module-path": "/home/dee/dev/rick-major-review/pdfbox/rick-major/"
    },
    {
        "mut": "getWidthFromFont(int)",
        "mut-start-line": 362,
        "mut-end-line": 372,
        "parent-module-path": "/home/dee/dev/rick-major-review/pdfbox/pdfbox/",
        "parent-file": "PDTrueTypeFont.java",
        "parent-fqn": "org.apache.pdfbox.pdmodel.font.PDTrueTypeFont",
        "parent-path": "/home/dee/dev/rick-major-review/pdfbox/pdfbox/src/main/java/org/apache/pdfbox/pdmodel/font/",
        "has-oo": True,
        "test-file": "TestPDTrueTypeFontRickGen.java",
        "test-path": "/home/dee/dev/rick-major-review/pdfbox/rick-major/src/test/java/org/apache/pdfbox/pdmodel/font/",
        "test-module-path": "/home/dee/dev/rick-major-review/pdfbox/rick-major/"   
    },
        {
        "mut": "initialize(Bootstrap)",
        "mut-start-line": 42,
        "mut-end-line": 50,
        "parent-module-path": "/home/dee/dev/rick-major-review/graphhopper/web/",
        "parent-file": "GraphHopperApplication.java",
        "parent-fqn": "com.graphhopper.application.GraphHopperApplication",
        "parent-path": "/home/dee/dev/rick-major-review/graphhopper/web/src/main/java/com/graphhopper/application/",
        "has-oo": False,
        "test-file": "TestGraphHopperApplicationRickGen.java",
        "test-path": "/home/dee/dev/rick-major-review/graphhopper/rick-major/src/test/java/com/graphhopper/application/",
        "test-module-path": "/home/dee/dev/rick-major-review/graphhopper/rick-major/"
    },
    {
        "mut": "setMaxDistance(double)",
        "mut-start-line": 55,
        "mut-end-line": 59,
        "parent-module-path": "/home/dee/dev/rick-major-review/graphhopper/core/",
        "parent-file": "DouglasPeucker.java",
        "parent-fqn": "com.graphhopper.util.DouglasPeucker",
        "parent-path": "/home/dee/dev/rick-major-review/graphhopper/core/src/main/java/com/graphhopper/util/",
        "has-oo": False,
        "test-file": "TestDouglasPeuckerRickGen.java",
        "test-path": "/home/dee/dev/rick-major-review/graphhopper/rick-major/src/test/java/com/graphhopper/util/",
        "test-module-path": "/home/dee/dev/rick-major-review/graphhopper/rick-major/"
    },
    {
        "mut": "loadExisting()",
        "mut-start-line": 48,
        "mut-end-line": 61,
        "parent-module-path": "/home/dee/dev/rick-major-review/graphhopper/core/",
        "parent-file": "StorableProperties.java",
        "parent-fqn": "com.graphhopper.storage.StorableProperties",
        "parent-path": "/home/dee/dev/rick-major-review/graphhopper/core/src/main/java/com/graphhopper/storage/",
        "has-oo": True,
        "test-file": "TestStorablePropertiesRickGen.java",
        "test-path": "/home/dee/dev/rick-major-review/graphhopper/rick-major/src/test/java/com/graphhopper/storage/",
        "test-module-path": "/home/dee/dev/rick-major-review/graphhopper/rick-major/"
    }
]

## Test runner

In [ ]:
def install_parent_module(target):
    os.chdir(target["parent-module-path"])
    process = subprocess.Popen(['mvn', 'clean', 'install', '-DskipTests'],
                               stdout=subprocess.PIPE, 
                               stderr=subprocess.PIPE)
    output_to_log("[LOG] Invoked a clean install of the parent module with {}".format(process.args))
    stdout, stderr = process.communicate()
    output_to_log(stdout.decode("utf-8"))
    output_to_log(stderr.decode("utf-8"))
    output_to_log("---------------------------------------------------")
    os.chdir(experiment_directory)

In [ ]:
def run_mvn_test(target):
    os.chdir(target["test-module-path"])
    output_to_log("[LOG] In test module directory " + os.getcwd())
    # run tests
    test_class = target["test-file"].replace(".java", "")
    run_test_class = "-Dtest=" + test_class
    process = subprocess.Popen(['mvn', 'test', run_test_class],
                               stdout=subprocess.PIPE, 
                               stderr=subprocess.PIPE)
    output_to_log("[LOG] Invoked tests with {}".format(process.args))
    stdout, stderr = process.communicate()
    output_to_log(stdout.decode("utf-8"))
    output_to_log(stderr.decode("utf-8"))
    
    output_to_log("---------------------------------------------------")

    # store result here
    result = re.search(r"(Tests run:.*)\n",
                       stdout.decode("utf-8")).group(1)
        
    process = subprocess.Popen(['mvn', 'clean'],
                               stdout=subprocess.PIPE, 
                               stderr=subprocess.PIPE)
    output_to_log("[LOG] Cleaned targets with {}".format(process.args))
    stdout, stderr = process.communicate()
    output_to_log(stdout.decode("utf-8"))
    output_to_log(stderr.decode("utf-8"))
    
    output_to_log("---------------------------------------------------")

    # cd back into experiment directory
    os.chdir(experiment_directory)
    output_to_log("[LOG] In experiment directory " + os.getcwd())
    output_to_log("---------------------------------------------------")
    
    return result

In [ ]:
def run_baseline_tests_with_no_oracles(target):
    output_to_log("[BASELINE] Not evaluating any oracle")
    # comment out all assertEquals and Mockito.verify statements"
    test_file_name = target["test-path"] + target["test-file"]
    output_to_log("[LOG] Modifying " + test_file_name)
    for line in fileinput.input(test_file_name, inplace = 1): 
        if oo_string in line or po_string in line or co_string in line:
            print(re.sub(r"^(.*)", "//\\1", line), end = '')
        else:
            print(line, end = '')
    
    # cd into test module, run tests, cd back into experiments directory
    result_baseline = run_mvn_test(target)
    
    # restore original test
    restore_original_RICK_test(target)
    output_to_log("---------------------------------------------------")
    return result_baseline

In [ ]:
def run_oo(target):
    output_to_log("[OO] Evaluating only OO")
    # OO: comment out all Mockito.verify statements
    test_file_name = target["test-path"] + target["test-file"]
    output_to_log("[LOG] Modifying " + test_file_name)
    for line in fileinput.input(test_file_name, inplace = 1): 
        if po_string in line or co_string in line:
            print(re.sub(r"^(.*)", "// \\1", line), end = '')
        else:
            print(line, end = '')
    
    # cd into test module, run tests, cd back into experiments directory
    result_oo = run_mvn_test(target)
    
    # restore original test
    restore_original_RICK_test(target)
    output_to_log("---------------------------------------------------")
    return result_oo

In [ ]:
def run_po(target):
    output_to_log("[PO] Evaluating only PO")
    # PO: comment out all assertEquals, Mockito.times() statements
    test_file_name = target["test-path"] + target["test-file"]
    output_to_log("[LOG] Modifying " + test_file_name)
    for line in fileinput.input(test_file_name, inplace = 1): 
        if oo_string in line or co_string in line:
            print(re.sub(r"^(.*)", "// \\1", line), end = '')
        else:
            print(line, end = '')
    
    # cd into test module, run tests, cd back into experiments directory
    result_po = run_mvn_test(target)
    
    # restore original test
    restore_original_RICK_test(target)
    output_to_log("---------------------------------------------------")
    return result_po

In [ ]:
def run_co(target):
    output_to_log("[CO] Evaluating only CO")
    # CO: comment out all assertEquals, Mockito.atLeastOnce() statements
    test_file_name = target["test-path"] + target["test-file"]
    output_to_log("[LOG] Modifying " + test_file_name)
    for line in fileinput.input(test_file_name, inplace = 1): 
        if oo_string in line or po_string in line:
            print(re.sub(r"^(.*)", "// \\1", line), end = '')
        else:
            print(line, end = '')
    
    # cd into test module, run tests, cd back into experiments directory
    result_co = run_mvn_test(target)
    
    # restore original test
    restore_original_RICK_test(target)
    output_to_log("---------------------------------------------------")
    return result_co

In [ ]:
def run_all_oracles(target):
    output_to_log("[ALL] Evaluating (OO +) PO + CO")
    # restore original tests
    restore_original_RICK_test(target)
    
    # cd into test module, run tests, cd back into experiments directory
    result_all = run_mvn_test(target)
    
    output_to_log("---------------------------------------------------")
    return result_all

## Util

In [ ]:
def output_to_log(message):
    with open(mut_mutation_log_path, 'a+') as logfile:
        logfile.write(message)
        logfile.write("\n")

In [ ]:
def cleanup_existing_results():
    output_to_log("[LOG] Removing pre-existing mutants from current working directory")
    os.system('rm -rf *.java/')
    output_to_log("[LOG] Cleaning up LittleDarwin results")
    os.system('rm -rf ' + littledarwin_results_path)

In [ ]:
def back_up_original_RICK_test(target):
    output_to_log("[LOG] Backing up original RICK test as ./" + target["parent-file"] + "/original-test.java")
    backup_rick_test = "cp " + target["test-path"] + target["test-file"] + " ./" + target["parent-file"] + "/original-test.java"
    output_to_log("[LOG] Invoking " + backup_rick_test)
    os.system(backup_rick_test)

In [ ]:
def restore_original_RICK_test(target):
    output_to_log("[LOG] Restoring original RICK test from ./" + target["parent-file"] + "/original-test.java")
    restore_rick_test = "cp" + " ./" + target["parent-file"] + "/original-test.java" + " " + target["test-path"] + target["test-file"]
    output_to_log("[LOG] Invoking " + restore_rick_test)
    os.system(restore_rick_test)

In [ ]:
def find_if_mutant_is_in_mut(target, mutant_file):
    search_string = "line number in original file: "
    found = ""
    with open(mutant_file, 'r') as fp:
        for l_no, line in enumerate(fp):
            # search string
            if search_string in line:
                found = line
                print("Processing mutant file", mutant_file)
                print("mutant found:", found)
                break
                
    result = int(re.search(r"(line number in original file:\s)(\d+)", found).group(2))
    return result >= target["mut-start-line"] and result <= target["mut-end-line"]

In [ ]:
def get_mutant_files(parent):
    mutants = ('./' + parent + '/[0-9]*.java')
    mutant_files = sorted(glob.glob(mutants))
    return mutant_files

In [ ]:
def copy_littledarwin_results(parent):
    output_to_log("[LOG] Copying mutants of " + parent)
    os.system('cp -r ' + littledarwin_results_path + parent + " ./")

In [ ]:
def run_littledarwin(path):
    output_to_log("[LOG] Running littledarwin on " + path)
    process = subprocess.Popen(['python3', '-m', 'littledarwin', '-m', '-v', '-p', path, '--all'],
                               stdout=subprocess.PIPE, 
                               stderr=subprocess.PIPE)
    output_to_log("[LOG] Invoked LitteDarwin with {}".format(process.args))
    stdout, stderr = process.communicate()
    output_to_log(stdout.decode("utf-8"))
    output_to_log(stderr.decode("utf-8"))

In [ ]:
def copy_mutant_files_to_output_directory(target):
    mutants_in_experiment_directory = experiment_directory + target["parent-file"]
    path_for_mutants = output_directory + target["mut"] + "/"
    output_to_log("[LOG] Copying littledarwin mutants to " + path_for_mutants)
    process = subprocess.Popen(['cp', '-r', mutants_in_experiment_directory, path_for_mutants],
                               stdout=subprocess.PIPE, 
                               stderr=subprocess.PIPE)
    output_to_log("[LOG] Invoked copying with {}".format(process.args))
    stdout, stderr = process.communicate()
    output_to_log(stdout.decode("utf-8"))
    output_to_log(stderr.decode("utf-8"))

In [ ]:
def run_experiment_with_mutant_file(target, mutant_file, original_file):
    output_to_log("[LOG] Replacing original" + target["parent-file"] + " with " + mutant_file)
    replace_original_with_mutant = "cp " + mutant_file + " " + target["parent-path"] + target["parent-file"]
    output_to_log("[LOG] Invoking " + replace_original_with_mutant)
    os.system(replace_original_with_mutant)
    
    # cd into parent module, make a clean install, cd back into experiments directory
    install_parent_module(target)
        
    target_report = {
        "mut": target["mut"],
        "CUT": target["parent-fqn"],
        "mutant-file": mutant_file,
        "mutant-within-mut": True
    }
    
    # test execution
    target_report["result_baseline"] = run_baseline_tests_with_no_oracles(target)
    if target["has-oo"]:
        target_report["result-oo"] = run_oo(target)
    else:
        target_report["result-oo"] = ""
    target_report["result-po"] = run_po(target)
    target_report["result-co"] = run_co(target)
    target_report["result-all"] = run_all_oracles(target)
    
    mutation_report_file = output_directory + target["mut"] + "/mutation-report.json"
    
    with open(mutation_report_file, 'a+') as outfile:
        json.dump(target_report, outfile, indent=4)
        outfile.write(",\n")
        
    output_to_log("[LOG] Resetting " + target["parent-file"] + " to original")
    replace_mutant_with_original = "cp " + original_file + " " + target["parent-path"] + target["parent-file"]
    output_to_log("[LOG] Invoking " + replace_mutant_with_original)
    os.system(replace_mutant_with_original)

## Mutation analysis

In [ ]:
def run_mutation_analysis(target):
    global mut_mutation_log_path
    mut_mutation_log_path = output_directory + target["mut"] + "/mutation-analysis.log"
    os.makedirs(os.path.dirname(mut_mutation_log_path), exist_ok=True)
    
    # housekeeping
    cleanup_existing_results()
    output_to_log("---------------------------------------------------")

    output_to_log("[LOG] Working with " + target["mut"])
    output_to_log("[LOG] Generating mutants for its parent " + target["parent-file"])
    output_to_log("---------------------------------------------------")
    
    # run littledarwin
    run_littledarwin(target["parent-path"])    
    output_to_log("---------------------------------------------------")
    
    # copy littledarwin results to current directory
    copy_littledarwin_results(target["parent-file"])
    output_to_log("---------------------------------------------------")

    # get list of mutant files
    mutant_files = get_mutant_files(target["parent-file"])
    output_to_log("[LOG] Generated " + str(len(mutant_files)) + " mutants")
    output_to_log("---------------------------------------------------")

    # get original file
    original_file = glob.glob(r'./' + target["parent-file"] + '/original.java')[0]
    
    # back-up original RICK test
    back_up_original_RICK_test(target)
    output_to_log("---------------------------------------------------")
    
    for f in range(len(mutant_files)):
        if find_if_mutant_is_in_mut(target, mutant_files[f]):
            run_experiment_with_mutant_file(target, mutant_files[f], original_file)
            output_to_log("---------------------------------------------------")
    
    # copy all mutants to output folder
    copy_mutant_files_to_output_directory(target)
    
    output_to_log("====== END OF THE LINE ======")

In [ ]:
def main():
    global experiment_directory
    global output_directory
    global littledarwin_results_path
    global oo_string
    global po_string
    global co_string

    experiment_directory = "/home/dee/dev/rick-major-review/finches/"
    output_directory = experiment_directory + "output/"
    littledarwin_results_path = "/home/dee/anaconda3/lib/python3.8/site-packages/littledarwin/LittleDarwinResults/"

    oo_string = "assertEquals"
    po_string = "Mockito.atLeastOnce"
    co_string = "Mockito.times"
    
    os.chdir(experiment_directory)
    targets_pdfbox = read_mut_data("./muts-pdfbox.json")
    targets_graphhopper = read_mut_data("./muts-graphhopper.json")
    targets_gephi = read_mut_data("./muts-gephi.json")

    print("Found", len(targets_pdfbox), "MUTs for PDFBox")
    print("Found", len(targets_graphhopper), "MUTs for GraphHopper" )
    print("Found", len(targets_gephi), "MUTs for Gephi" )
    
    print("Working with PDFBox MUTs")
    for target in targets_pdfbox:
        print(target["mut"])
        run_mutation_analysis(target)
    
    print("Working with GraphHopper MUTs")
    for target in targets_graphhopper:
        print(target["mut"])
        run_mutation_analysis(target)
    
    print("Working with Gephi MUTs")
    for target in targets_gephi:
        print(target["mut"])
        run_mutation_analysis(target)
    
    # run_mutation_analysis(targets[5])

In [ ]:
main()